# Introducción

En el siguiente proyecto estaré desarrollando un modelo de aprendizaje para un banco que está experimentando una creciente tasa de abandono por parte de sus clientes cada mes.  Para el banco es más económico salvar a los clientes que permanecen, que atrer a nuevos. 

Mi objetivo es crear un modelo de clasificación que me permita predecir exitosamente si un cliente abandonará el banco pronto. Para cumplir con esa tarea, estaré trabajando con la base de datos proporcionada por el banco, la cual me permitirá evaluar el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Mi enfoque en cuanto a las predicciones de si un cliente abandonará o no, será disminuir el máximo posible de falsos negativos y falsos positivos; para lo cual me concentraré en la media armónica de las métricas de precisión y sensibilidad.

#### Tabla de contenido:

1. Inicialización: importar las librerías.
  - 1.2. Cargar y preparar los datos. 
2. Codificación de los datos categóricos:
  - 2.2. Codificación OHE, segmentación de conjuntos de datos y estandarización de características numéricas para modelos de regresión.
  - 2.3. Codificación de etiquetas, segmentación de conjuntos de datos y estandarización de características numéricas  para modelos basados en árboles.
3. Entrenar los modelos.
  - 3.2. Examinar el equilibrio de clases.
4. Elegir un modelo y comprobar su calidad con el conjunto de prueba.
5. Prueba de cordura en el modelo elegido.
6. Conclusiones generales.

# 1. Inicialización

In [1]:
#importar librerías
import pandas as pd
from sklearn.metrics import recall_score, precision_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.utils import shuffle
import matplotlib as plt

#librerias para los modelos 
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1.2. Cargar y preparar los datos 

In [2]:
#cargar el dataset en un dataframe
data = pd.read_csv('dataset/Churn.csv')

In [3]:
#visualizar información general del dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
#visualizar los datos
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


En cuanto a los tipos de datos de cada columna, parecen correctos para cada categoría correspondiente. Sin embargo, para mi tarea de clasificación debo codificar más adelante algunas columnas categóricas; estas columnas serán: 'Surname', 'Geography' y 'Gender'.

In [5]:
#verificar los duplicados en el dataframe
data.duplicated().sum()

0

In [6]:
#verificar datos ausentes
data.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

He encontrado algunos datos ausentes en la columna 'Tenure'. Esta columna contiene el período durante el cual ha madurado el depósito a plazo fijo de un cliente (años).
En vista de que esa información pudiera ser importante para que el modelo pueda predecir si un cliente abandonará el banco, procederé a rellenar los datos ausentes con el período mediano, agrupando por país y género.

In [7]:
#rellenar los datos ausentes en la columna 'Tenure'
data['Tenure'] = data['Tenure'].fillna(data.groupby(['Geography', 'Gender'])['Tenure'].transform('median'))

#verificar nuevamente los datos ausentes
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
#Eliminar columnas innecesarias del DataFrame
set_data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Preparé un DataFrame, eliminando los datos ausentes y las columnas innecesarias para realizar las predicciones.
Los datos ya están preparados para pasar a la segmentación de los conjuntos de entrenamiento, validación y prueba.

# 2. Codificación de columnas categóricas

### 2.1. Codificación OHE de columnas categóricas para los modelos de regresión

In [9]:
#codificación OHE para las regresiones
data_ohe = pd.get_dummies(set_data, drop_first=True)

#definir el objetivo y las features
target_ohe = data_ohe['Exited']
features_ohe = data_ohe.drop(['Exited'], axis=1)

#segmentación de los datos para conjuntos de entrenamiento y validación ohe
features_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_test_split(features_ohe, target_ohe, test_size=0.40, random_state=12345)

#segmentacion del conjunto de validacion para conjunto de prueba ohe
features_valid_ohe, features_test_ohe, target_valid_ohe, target_test_ohe = train_test_split(features_valid_ohe, target_valid_ohe, test_size=0.40, random_state=12345)

#verificando los tamaños de los conjuntos 
print('Tamaños de los conjuntos de datos OHE:')
print('features train ohe:', features_train_ohe.shape)
print('target train ohe:', target_train_ohe.shape)
print()
print('Conjuntos de validación:')
print('features valid ohe:', features_valid_ohe.shape)
print('target_valid ohe:', target_valid_ohe.shape)
print()
print('Conjuntos de prueba:')
print('features test ohe:', features_test_ohe.shape)
print('target test ohe:', target_test_ohe.shape)

Tamaños de los conjuntos de datos OHE:
features train ohe: (6000, 11)
target train ohe: (6000,)

Conjuntos de validación:
features valid ohe: (2400, 11)
target_valid ohe: (2400,)

Conjuntos de prueba:
features test ohe: (1600, 11)
target test ohe: (1600,)


Para los modelos de regresión, he codificado las columnas categóricas del DataFrame utilizando una codificación One-Hot. A continuación, segmenté el dataset baset en tres conjuntos: entrenamiento, validación y prueba, en una proporción de 6:4.4.

### 2.2. Codificación de etiquetas para modelos basados en árboles

In [10]:
#ordinal encoder para modelos basados en árboles
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(set_data), columns=set_data.columns)

#definir el objetivo y las features 
target_ordinal = data_ordinal['Exited']
features_ordinal = data_ordinal.drop(['Exited'], axis=1)

#segmentar los datos en conjuntos de entrenamiento y validcación
features_train_ordinal, features_valid_ordinal, target_train_ordinal, target_valid_ordinal = train_test_split(features_ordinal, target_ordinal, test_size=0.40, random_state=12345)

#segmentar el conjunto de validación en un subconjunto de prueba
features_valid_ordinal, features_test_ordinal, target_valid_ordinal, target_test_ordinal = train_test_split(features_valid_ordinal, target_valid_ordinal, test_size=0.40, random_state=12345)


print('Tamaños de los conjuntos de datos:')
print('features train ordinal:' , features_train_ordinal.shape)
print('target train ordinal:' , target_train_ordinal.shape)
print()
print('features valid ordinal:' , features_valid_ordinal.shape)
print('target valid ordinal:' , target_valid_ordinal.shape)
print()
print('features test ordinal:', features_test_ordinal.shape)
print('target test ordinal:', target_test_ordinal.shape)


Tamaños de los conjuntos de datos:
features train ordinal: (6000, 10)
target train ordinal: (6000,)

features valid ordinal: (2400, 10)
target valid ordinal: (2400,)

features test ordinal: (1600, 10)
target test ordinal: (1600,)


Para los modelos basados en árboles que voy a entrenar, decidí codificar las columnas categóricas utilizando la codificación de etiquetas, más especificamente, el OrdinalEncoder. Luego, también segmenté el dataset base en tres conjuntos: entrenamiento, validación y prueba, en una proporción 6:4:4. 

## 3. Examinar el equilibrio de clases

In [11]:
#examinar el equilibrio de clases 
print('Conteo de objetivos de clase positiva (1):', (set_data['Exited'] == 1).sum())
print('Conteo de objetivos de clase negativa (0)":', (set_data['Exited'] == 0).sum())

Conteo de objetivos de clase positiva (1): 2037
Conteo de objetivos de clase negativa (0)": 7963


Al examinar la variable objetivo, se observa un fuerte desequilibrio entre la clase 1 y la clase 0.

## 4. Entrenar modelos

### 4.1. Modelo basado en árboles de decisión

In [12]:
#conjuntos de entrenamiento 
features_train_ordinal
target_train_ordinal

#conjuntos de validación
features_valid_ordinal
target_valid_ordinal

#conjuntos de prueba
features_test_ordinal
target_test_ordinal

#arbol de decision
best_model_dt = None
best_depth_dt = 0
f1_best_result_dt = 0.59

for depth in range (1, 10):
    model_tree = DecisionTreeClassifier(
        random_state=12345, max_depth=depth, min_samples_split=3, min_samples_leaf=3
        )
    model_tree.fit(features_train_ordinal, target_train_ordinal)
    
    predicted_valid = model_tree.predict(features_valid_ordinal)
    result = f1_score(target_valid_ordinal, predicted_valid)

    if result >= f1_best_result_dt:
        best_model_dt = model_tree
        f1_best_result_dt = result
        best_depth_dt = depth



print('Mejor modelo de árbol de decisión:', best_model_dt)
print('Mejor valor f1 para árbol de decisión:', f1_best_result_dt)

Mejor modelo de árbol de decisión: None
Mejor valor f1 para árbol de decisión: 0.59


### 4.2.. Modelo de bosque aleatorio

In [13]:
#conjuntos de entrenamiento 
features_train_ordinal
target_train_ordinal

#conjuntos de validación
features_valid_ordinal
target_valid_ordinal

#conjuntos de prueba
features_test_ordinal
target_test_ordinal

#bosque aleatorio
best_model_f = None
f1_best_result_f = 0.59
best_depth_f = 0
best_est_f = 0

for est in range(10, 80, 100):
    for depth in range(1, 20):
        model_forest = RandomForestClassifier(
            random_state=54321, n_estimators=est, max_depth=depth, min_samples_leaf=3, 
            min_samples_split=2, max_features=5
            )
        model_forest.fit(features_train_ordinal, target_train_ordinal)
        predicted_valid_forest = model_forest.predict(features_valid_ordinal)
        
        result = f1_score(target_valid_ordinal, predicted_valid_forest)

        if result >= f1_best_result_f:
            best_model_f = model_forest
            f1_best_result_f = result
            best_depth_f = depth
            best_est_f = est

print('Mejor modelo de bosque aleatorio:', best_model_f)
print()
print('Mejor valor f1 para bosque aleatorio:', f1_best_result_f)



Mejor modelo de bosque aleatorio: None

Mejor valor f1 para bosque aleatorio: 0.59


### 4.3. Modelo de regresión logística

In [14]:
#Conjuntos de entrenamiento
features_train_ohe
target_train_ohe

#Conjuntos de validación
features_valid_ohe
target_valid_ohe

#Conjunto de prueba 
features_test_ohe
target_test_ohe

#regresión logística
model_logistic_regre = LogisticRegression(random_state=12345, solver='liblinear', max_iter=800, penalty='l2')
model_logistic_regre.fit(features_train_ohe, target_train_ohe)

predicted_valid_logistic_regre = model_logistic_regre.predict(features_valid_ohe)

f1_logistic_regresion = f1_score(target_valid_ohe, predicted_valid_logistic_regre)

print('Valor f1 para regresión logística:', f1_logistic_regresion)


Valor f1 para regresión logística: 0.09540636042402827


He entrenado tres modelos de clasificación: árboles de decisión, bosque aleatorio y regresión logística. Al momento de entrenarlos no he tomado en cuenta el desequilibrio de clases existente, por lo tanto, ninguno de los modelos me ha proporcionado un valor F1 suficientemente alto (de al menos 0.59), como para que resulten útiles en mi tarea..
A continuación utilizaré dos enfoques para equilibrar las clases y mejoraré la calidad de los modelos. 

## 5. Corregir el desequilibrio de clases

### 5.1. Sobremuestreo (upsample)

In [15]:
#sobremuestreo (upsample)
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    repeat = 10
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled


In [32]:
#definir variables sobremuestreadas para modelos de arboles
features_train_upsampled_trees, target_train_upsampled_trees = upsample(features_train_ordinal, target_train_ordinal, 10)

#definir variables sobremuestreadas para modelo de regresión
features_train_upsampled_regre, target_train_upsampled_regre = upsample(features_train_ohe, target_train_ohe, 10)

He definido cuatro variables de entrenamiento sobremuestreadas, para los modelos de árboles y el de regresión respectivamente.

### 5.2. Submuestreo (downsample)

In [22]:
#submuestreo (downsample)
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_upsampled, target_upsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled


In [33]:
#definir variables submuestreadas para modelos de arboles
features_train_downsampled_trees, target_train_downsampled_trees = downsample(features_train_ordinal, target_train_ordinal, 0.1)

#definir variables submuestreadas para modelo de regresión
features_train_downsampled_regre, target_train_downsampled_regre = downsample(features_train_ohe, target_train_ohe, 0.1)

## 6. Mejorar la calidad de los modelos con clases equilibradas

### 6.1 Árbol de decisión

In [46]:
#modelo basado en árboles de decisión con clases equilibradas

best_model_tree = None
f1_best_result_tree = 0.30

for depth in range (1, 10):
    model_tree = DecisionTreeClassifier(
        random_state=12345, max_depth=depth, min_samples_split=3, min_samples_leaf=3, class_weight='balanced'
        )
    model_tree.fit(features_train_upsampled_trees, target_train_upsampled_trees)
    
    predicted_valid = model_tree.predict(features_valid_ordinal)
    result = f1_score(target_valid_ordinal, predicted_valid)

    if result >= f1_best_result_tree:
        best_model_tree = model_tree
        f1_best_result_tree = result
    

print('Mejor modelo de árbol de decisión:', best_model_tree)
print('Mejor valor f1 para árbol de decisión:', f1_best_result_tree)


Mejor modelo de árbol de decisión: DecisionTreeClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=3,
                       min_samples_split=3, random_state=12345)
Mejor valor f1 para árbol de decisión: 0.5884476534296029


### 6.2. Bosque aleatorio

In [64]:
#bosque aleatorio
best_model_forest = None
f1_best_result_forest = 0.59

for est in range(10, 80, 100):
    for depth in range(1, 20):
        model_forest = RandomForestClassifier(
            random_state=54321, n_estimators=est, max_depth=depth, min_samples_leaf=3, 
            min_samples_split=3, max_features=5, class_weight='balanced'
            )
        model_forest.fit(features_train_upsampled_trees, target_train_upsampled_trees)
        predicted_valid_forest = model_forest.predict(features_valid_ordinal)
        
        result = f1_score(target_valid_ordinal, predicted_valid_forest)

        if result >= f1_best_result_forest:
            best_model_forest = model_forest
            f1_best_result_forest = result


print('Mejor modelo de bosque aleatorio:', best_model_forest)
print()
print('Mejor valor f1 para bosque aleatorio:', f1_best_result_forest)


Mejor modelo de bosque aleatorio: RandomForestClassifier(class_weight='balanced', max_depth=19, max_features=5,
                       min_samples_leaf=3, min_samples_split=3, n_estimators=10,
                       random_state=54321)

Mejor valor f1 para bosque aleatorio: 0.5935984481086324
